In [255]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import statistics

from konlpy.tag import Twitter
from konlpy.utils import pprint
twitter = Twitter()
from collections import Counter
from collections import namedtuple

pd.set_option('max_colwidth', 500)

common_front = '../../dataset/go_0715/gogosing_' 
common_back = '.json'

def get_file(file_num):   
    file_path = common_front + str(file_num) + common_back
    json_data = open(file_path).read()
    data = json.loads(json_data)
    df_data = pd.DataFrame(data)
    return df_data


# 74번 파일이 에러가 나서 제외함
def concat_file(file_num):
    start_file = pd.DataFrame(get_file(1))
    for i in range(2,file_num+1):
        if( i == 74):
            continue
        df_tmp = pd.DataFrame(get_file(i))
        start_file = pd.concat([start_file, df_tmp])
    start_file = start_file.reset_index(drop=True)
    return start_file

In [3]:
go_data = concat_file(156)

# Practice

In [215]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Twitter

In [11]:
data = ["I love machine learning. Its awesome.",
       "I love coding in python",
       "I love building cahtbots",
       "they chat amagingly well"]

In [216]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

Training

In [185]:
max_epochs = 100
vec_size = 200
alpha = 0.025

model = Doc2Vec(size=vec_size,      # Dimensionality of the feature vectors
                window=8,            # The maximum distance 
                alpha=alpha,        # initial learning-rate
                min_alpha=0.00025,  # min learning-rate
                min_count=1,        # Ignores all words with total frequency lower than this.
                dm =1)              # pv-dm
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Model Saved


Test

In [80]:
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
print("tokenize", test_data)
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('2')
print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs[1])

('tokenize', ['i', 'love', 'chatbots'])
('V1_infer', array([-0.01693995,  0.01443483, -0.01117608, -0.00651255, -0.00140172,
        0.00699042, -0.01420608,  0.00381565, -0.00207753, -0.00611731,
        0.01641728,  0.01888668,  0.01387223,  0.02155275, -0.00276469,
       -0.0159132 ,  0.00120709, -0.01078141,  0.00982879, -0.00869922],
      dtype=float32))
[('1', 0.9952065944671631), ('0', 0.9928261041641235), ('3', 0.9872888326644897)]
[ 0.12823918 -0.36749974 -0.08225223 -0.3351773   0.2147343   0.17162144
  0.201036   -0.41579503 -0.32589528 -0.04822816  0.02442648 -0.3198524
  0.23017946 -0.2422444   0.31409585 -0.21874669 -0.18283987 -0.02778132
  0.33491543  0.19073087]


In [38]:
doc_vectorizer = Doc2Vec.load('d2v.model')

In [41]:
print(doc_vectorizer.wv.most_similar('machine'))

[('love', 0.9588698148727417), ('chat', 0.9288003444671631), ('i', 0.9260274171829224), ('learning', 0.9186973571777344), ('.', 0.9120382070541382), ('awesome', 0.9076647162437439), ('its', 0.9057942628860474), ('building', 0.9057425260543823), ('they', 0.8862553238868713), ('amagingly', 0.8675417900085449)]


# apply to gogosing data

In [223]:
num_cId = go_data['cId'].value_counts()
num_cId = pd.DataFrame(num_cId)
print(num_cId)

                      cId
네이버 페이 구매자          52539
rkdldk5**             475
yeji04**              449
elqxhs**              339
dnv**                 314
rladmswl9708**        308
wlgus5**              305
pp**                  301
wertyuio9**           292
sumin12**             278
dldmsql10**           276
kma05**               271
dream3**              245
rudgml72**            244
soa12**               237
kimhani3**            210
45216058**            209
sjj44**               197
rlaqhfk5**            193
tladnjs2**            191
tpwls12**             190
jy03**                189
egll**                188
jeje05**              187
bha11**               186
hooming**             186
tofha40**             186
ds357**               185
miliastla**           183
phr9101**             183
...                   ...
유민지                     1
461301292**             1
gook**                  1
byeolttar**             1
1698601763758607**      1
tjddlaks1**             1
eofhdrk**   

In [224]:
cIds = num_cId.drop(num_cId.index[0])  #drop naverpay
cIds = cIds[cIds['cId']!=1]  #drop count1
cIds_list = cIds.index.tolist()
len(cIds_list)

92802

# psbp** doc2vec 적용

In [276]:
df=(go_data.loc[go_data['cId']=='psbp**', ['cId','desc']])
print df['desc']

43527                                                                                                                                                                        조금 긴 슬랙스를 찾다가 고고싱 신상에 올라왔길래 바로 주문했어요평소 26입는데 s사이즈 조금 헐렁한데 괜찮아요근데 이렇게 얇은 재질인줄은 몰랐는데.. 여름에도 입을 수 있겠어요~~
43529                                                                                                                                                                                                                    데일리로 입을 흰 블라우스 찾다가 샀어요!저렴한 가격에 잘 산거 같아요 스판도 좋아서 불편하지않아요:)
43530                                                                                                                                                                                                                           흰색 나시가 필요해서 다른 옷 살때 나시도 같이 샀어요~이름처럼 스판 쫀쫀해서 좋아요ㅎㅎㅎ
102222                                                                                                                                                                         

In [277]:
len(df['desc'])

56

In [287]:
def tagging(data):
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(df['desc'])]
    return tagged_data

In [294]:
def tokenizer_twitter_morphs(doc):
    tagged_data = [TaggedDocument(words= twitter.morphs(doc), tags=[str(i)]) for i, _d in enumerate(df['desc'])]
    return tagged_data

In [295]:
df['token_review'] = df['desc'].apply(tokenizer_twitter_morphs)
df

,cId,desc,token_review
43527,psbp**,조금 긴 슬랙스를 찾다가 고고싱 신상에 올라왔길래 바로 주문했어요평소 26입는데 s사이즈 조금 헐렁한데 괜찮아요근데 이렇게 얇은 재질인줄은 몰랐는데.. 여름에도 입을 수 있겠어요~~,"[([조금, 긴, 슬랙스, 를, 찾다, 가, 고고싱, 신상, 에, 올라왔, 길래, 바로, 주문했, 어요, 평소, 26, 입는, 데, s, 사이즈, 조금, 헐렁한데, 괜찮, 아요, 근데, 이렇게, 얇은, 재질, 인, 줄, 은, 몰랐, 는데, .., 여름, 에도, 입, 을, 수, 있겠, 어요, ~~], [0]), ([조금, 긴, 슬랙스, 를, 찾다, 가, 고고싱, 신상, 에, 올라왔, 길래, 바로, 주문했, 어요, 평소, 26, 입는, 데, s, 사이즈, 조금, 헐렁한데, 괜찮, 아요, 근데, 이렇게, 얇은, 재질, 인, 줄, 은, 몰랐, 는데, .., 여름, 에도, 입, 을, 수, 있겠, 어요, ~~], [1]), ([조금, 긴, 슬랙스, 를, 찾다, 가, 고고싱, 신상, 에, 올라왔, 길래, 바로, 주문했, 어요, 평소, 26, 입는, 데, s, 사이즈, 조금, 헐렁한데, 괜찮, 아요, 근데, 이렇게, 얇은, 재질, 인, 줄, 은, 몰랐, 는데, .., 여름, 에도, 입, 을..."
43529,psbp**,데일리로 입을 흰 블라우스 찾다가 샀어요!저렴한 가격에 잘 산거 같아요 스판도 좋아서 불편하지않아요:),"[([데, 일리, 로, 입, 을, 흰, 블라우스, 찾다, 가, 샀, 어요, !, 저렴, 한, 가격, 에, 잘, 산, 거, 같아, 요, 스, 판도, 좋, 아서, 불편하지, 않아, 요, :)], [0]), ([데, 일리, 로, 입, 을, 흰, 블라우스, 찾다, 가, 샀, 어요, !, 저렴, 한, 가격, 에, 잘, 산, 거, 같아, 요, 스, 판도, 좋, 아서, 불편하지, 않아, 요, :)], [1]), ([데, 일리, 로, 입, 을, 흰, 블라우스, 찾다, 가, 샀, 어요, !, 저렴, 한, 가격, 에, 잘, 산, 거, 같아, 요, 스, 판도, 좋, 아서, 불편하지, 않아, 요, :)], [2]), ([데, 일리, 로, 입, 을, 흰, 블라우스, 찾다, 가, 샀, 어요, !, 저렴, 한, 가격, 에, 잘, 산, 거, 같아, 요, 스, 판도, 좋, 아서, 불편하지, 않아, 요, :)], [3]), ([데, 일리, 로, 입, 을, 흰, 블라우스, 찾다, 가, 샀, 어요, !, 저렴,..."
43530,psbp**,흰색 나시가 필요해서 다른 옷 살때 나시도 같이 샀어요~이름처럼 스판 쫀쫀해서 좋아요ㅎㅎㅎ,"[([흰색, 나시, 가, 필요해서, 다른, 옷, 살때, 나시, 도, 같이, 샀, 어요, ~, 이름, 처럼, 스판, 쫀쫀, 해서, 좋, 아요, ㅎㅎㅎ], [0]), ([흰색, 나시, 가, 필요해서, 다른, 옷, 살때, 나시, 도, 같이, 샀, 어요, ~, 이름, 처럼, 스판, 쫀쫀, 해서, 좋, 아요, ㅎㅎㅎ], [1]), ([흰색, 나시, 가, 필요해서, 다른, 옷, 살때, 나시, 도, 같이, 샀, 어요, ~, 이름, 처럼, 스판, 쫀쫀, 해서, 좋, 아요, ㅎㅎㅎ], [2]), ([흰색, 나시, 가, 필요해서, 다른, 옷, 살때, 나시, 도, 같이, 샀, 어요, ~, 이름, 처럼, 스판, 쫀쫀, 해서, 좋, 아요, ㅎㅎㅎ], [3]), ([흰색, 나시, 가, 필요해서, 다른, 옷, 살때, 나시, 도, 같이, 샀, 어요, ~, 이름, 처럼, 스판, 쫀쫀, 해서, 좋, 아요, ㅎㅎㅎ], [4]), ([흰색, 나시, 가, 필요해서, 다른, 옷, 살때, 나시, 도, 같이, 샀, 어..."
102222,psbp**,"예뻐서 샀는데 차라리 다른걸 살걸 그랬네요,,니트 넣어서 입었는데 뚱뚱해보여요 치마 길이도 앞에가 조금 더 길어요벨트도 다른걸 써야할것 같아요","[([예뻐, 서, 샀, 는데, 차라리, 다른, 걸, 살걸, 그랬, 네요, ,,, 니트, 넣어, 서, 입었, 는데, 뚱뚱해, 보여, 요, 치마, 길이, 도, 앞, 에가, 조금, 더, 길, 어요, 벨트, 도, 다른, 걸, 써, 야할, 것, 같아, 요], [0]), ([예뻐, 서, 샀, 는데, 차라리, 다른, 걸, 살걸, 그랬, 네요, ,,, 니트, 넣어, 서, 입었, 는데, 뚱뚱해, 보여, 요, 치마, 길이, 도, 앞, 에가, 조금, 더, 길, 어요, 벨트, 도, 다른, 걸, 써, 야할, 것, 같아, 요], [1]), ([예뻐, 서, 샀, 는데, 차라리, 다른, 걸, 살걸, 그랬, 네요, ,,, 니트, 넣어, 서, 입었, 는데, 뚱뚱해, 보여, 요, 치마, 길이, 도, 앞, 에가, 조금, 더, 길, 어요, 벨트, 도, 다른, 걸, 써, 야할, 것, 같아, 요], [2]), ([예뻐, 서, 샀, 는데, 차라리, 다른, 걸, 살걸, 그랬, 네요, ,,, 니트, 넣어, 서, 입었, 는..."
102223,psbp**,배송 빨라요~ 옷은 예쁜데 겨울에 입기에는 조금 얇아요.색도 사진이랑 똑같아요 완전 흰바지보다 아이보리라 마음에 들어요,"[([배송, 빨라, 요, ~, 옷, 은, 예쁜, 데, 겨울, 에, 입기, 에는, 조금, 얇아, 요, ., 색도, 사진, 이랑, 똑같, 아요, 완전, 흰, 바지, 보다, 아이보리, 라, 마음, 에, 들어, 요], [0]), ([배송, 빨라, 요, ~, 옷, 은, 예쁜, 데, 겨울, 에, 입기, 에는, 조금, 얇아, 요, ., 색도, 사진, 이랑, 똑같, 아요, 완전, 흰, 바지, 보다, 아이보리, 라, 마음, 에, 들어, 요], [1]), ([배송, 빨라, 요, ~, 옷, 은, 예쁜, 데, 겨울, 에, 입기, 에는, 조금, 얇아, 요, ., 색도, 사진, 이랑, 똑같, 아요, 완전, 흰, 바지, 보다, 아이보리, 라, 마음, 에, 들어, 요], [2]), ([배송, 빨라, 요, ~, 옷, 은, 예쁜, 데, 겨울, 에, 입기, 에는, 조금, 얇아, 요, ., 색도, 사진, 이랑, 똑같, 아요, 완전, 흰, 바지, 보다, 아이보리, 라, 마음, 에, 들어, 요], [3]), ([배..."
102224,psbp**,배송도 이틀만에 오고 옷도 예뻐요. 브라운이랑 블랙 고민했는데 마음에 들어요ㅋㅋ팔은 좀 길어서 한번 더 접어 입는게 더 예뻐요,"[([배송, 도, 이틀, 만에, 오고, 옷, 도, 예뻐, 요, ., 브라운, 이랑, 블랙, 고민했, 는데, 마음, 에, 들어, 요, ㅋㅋ, 팔, 은, 좀, 길, 어서, 한번, 더, 접어, 입는, 게, 더, 예뻐, 요], [0]), ([배송, 도, 이틀, 만에, 오고, 옷, 도, 예뻐, 요, ., 브라운, 이랑, 블랙, 고민했, 는데, 마음, 에, 들어, 요, ㅋㅋ, 팔, 은, 좀, 길, 어서, 한번, 더, 접어, 입는, 게, 더, 예뻐, 요], [1]), ([배송, 도, 이틀, 만에, 오고, 옷, 도, 예뻐, 요, ., 브라운, 이랑, 블랙, 고민했, 는데, 마음, 에, 들어, 요, ㅋㅋ, 팔, 은, 좀, 길, 어서, 한번, 더, 접어, 입는, 게, 더, 예뻐, 요], [2]), ([배송, 도, 이틀, 만에, 오고, 옷, 도, 예뻐, 요, ., 브라운, 이랑, 블랙, 고민했, 는데, 마음, 에, 들어, 요, ㅋㅋ, 팔, 은, 좀, 길, 어서, 한번, 더, 접어, 입는, 게, ..."
105567,psbp**,비슷한 신발이 있었는데 230신다가 조금 널널하게 신고싶어서 235로 주문했는데 진짜 가볍고 가격대비 만족해요 냄새도 별로 안나요,"[([비슷한, 신발, 이, 있었, 는데, 230, 신다, 가, 조금, 널널, 하게, 신고, 싶, 어서, 235, 로, 주문했, 는데, 진짜, 가볍, 고, 가격, 대비, 만족해, 요, 냄새, 도, 별로, 안나, 요], [0]), ([비슷한, 신발, 이, 있었, 는데, 230, 신다, 가, 조금, 널널, 하게, 신고, 싶, 어서, 235, 로, 주문했, 는데, 진짜, 가볍, 고, 가격, 대비, 만족해, 요, 냄새, 도, 별로, 안나, 요], [1]), ([비슷한, 신발, 이, 있었, 는데, 230, 신다, 가, 조금, 널널, 하게, 신고, 싶, 어서, 235, 로, 주문했, 는데, 진짜, 가볍, 고, 가격, 대비, 만족해, 요, 냄새, 도, 별로, 

In [308]:
def training_pvdm(tagged_data):
    max_epochs = 100
    vec_size = 20
    alpha = 0.025
    
    model = Doc2Vec(size=vec_size,      # Dimensionality of the feature vectors
                    window=8,            # The maximum distance 
                    alpha=alpha,        # initial learning-rate
                    min_alpha=0.00025,  # min learning-rate
                    min_count=1,        # Ignores all words with total frequency lower than this.
                    dm =1)              # pv-dm

    model.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        #print('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha

    model.save("gogosing_d2v.model")
    print("Model Saved")
    
    return model

In [309]:
def similarity(model):
    sm = []
    for i in range(len(model.docvecs)-1):
        for j in range(i+1, len(model.docvecs)):
            sm.append(model.docvecs.similarity(i,j))
        
    print str('최대값: ') + str(max(sm))
    print str('상위25%: ') + str(np.percentile(sm,75))
    print str('중간값: ') + str(np.percentile(sm,50))
    print str('상위75%: ') + str(np.percentile(sm,25))
    print str('평균: ') + str(np.mean(sm))
    
    return sm

twitter

In [310]:
tagged_data = tokenizer_twitter_morphs('psbp**')
model = training(tagged_data)
sm = similarity(model)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Model Saved
최대값: 0.8328396897953004
상위25%: 0.46729656940497166
중간값: 0.3361999315199555
상위75%: 0.1924911702966979
평균: 0.325525812002637


In [311]:
similar_doc = model.docvecs.most_similar('1')
print similar_doc

[('2', 0.6264384984970093), ('38', 0.6127101182937622), ('41', 0.6083108186721802), ('33', 0.5980275869369507), ('46', 0.5358717441558838), ('51', 0.5145502090454102), ('47', 0.5115160942077637), ('30', 0.49712538719177246), ('23', 0.4833873510360718), ('7', 0.45428138971328735)]


In [312]:
model.docvecs.similarity('1','25')

0.3663768121110114

word_tokenizer

In [305]:
tagged_data = tagging('psbp**')
model = training(tagged_data)
sm = similarity(model)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Model Saved
최대값: 0.9860137656702805
상위25%: 0.5102301315124822
중간값: 0.4280656221301966
상위75%: 0.335649792647146
평균: 0.42342009249681173


In [306]:
similar_doc = model.docvecs.most_similar('1')
print similar_doc

[('25', 0.6840797662734985), ('28', 0.6043823957443237), ('0', 0.5915035605430603), ('46', 0.5776281952857971), ('52', 0.5724379420280457), ('53', 0.5682433247566223), ('47', 0.5600717067718506), ('35', 0.55898118019104), ('17', 0.5577130913734436), ('41', 0.5554500222206116)]


In [307]:
model.docvecs.similarity('1','25')

0.6840797141153355

In [269]:
len(model.docvecs)

56

In [271]:
similarity(model)

최대값: 0.9860137656702805
상위25%: 0.5102301315124822
중간값: 0.4280656221301966
상위75%: 0.335649792647146
평균: 0.42342009249681173


[0.5915035345486221,
 0.4008606831252173,
 0.28749842713676166,
 0.49589175905534527,
 0.31923397508946105,
 0.6284063385114741,
 0.4952220203718668,
 0.5466674704202222,
 0.5305195497640144,
 0.2612060500477248,
 0.5200289561500834,
 0.3608699840390075,
 0.22641904662254184,
 0.3917996734976293,
 0.4920068401907595,
 0.2677738917153564,
 0.5202773622366779,
 0.45025981961648615,
 0.44697403135328173,
 0.5101072070441292,
 0.16747642323061324,
 0.23295437084269105,
 0.38019756560022083,
 0.472890743639772,
 0.5153235202776841,
 0.35458938005922136,
 0.4660319727874785,
 0.4067751415247944,
 0.559834616566485,
 0.2725146384761452,
 0.5216821525401807,
 0.460995706357983,
 0.3012029127593128,
 0.2343415580896973,
 0.3205596006917958,
 0.5634778184599557,
 0.49352921182963405,
 0.3090091828196391,
 0.3103251547753515,
 0.46347982369928964,
 0.3297692689253096,
 0.5588338719832878,
 0.2923807089369481,
 0.3761444939195433,
 0.38942131022021453,
 0.5253621662135856,
 0.32623054746635244,
 0

In [144]:
 # 소수점 아래 3자리까지 나타내기, 4째자리에서 반올림
def short_float(val):
    value = float("{:.4f}".format(val))
    return value


In [145]:
def print_want_val(sm_list,used_func):
    val=(used_func(sm_list))
    val=short_float(val)
    
    return val

# random 100명

In [173]:
import random

In [174]:
training_cId = random.sample(cIds_list, 100)
len(training_cId)
training_cId[:10]

[u'gombi**',
 u'sm81**',
 u'mj75**',
 u'cnh70**',
 u'sjmhshym**',
 u'sck090**',
 u'sjy95**',
 u'882663498512851**',
 u'yuhye8**',
 u'yeam**']

In [187]:
max_list=[]
mid_list=[]
q1_list=[]
q3_list=[]

for i in range(len(training_cId)):
    print str(i+1) + str('번째 리뷰어: ')+ str(training_cId[i].encode('utf-8'))
    df=(go_data.loc[go_data['cId']==training_cId[i], ['cId','desc']])
    desc_per_cId = df["desc"].tolist()
    tagged_data = tagging(desc_per_cId)
    model = training(tagged_data)
    sm = similarity(model)
    
    result_max= print_want_val(sm, lambda x: np.percentile(sm,100)) # 최대값 반환 
    max_list.append(result_max)
       
    result_q3 = print_want_val(sm, lambda x: np.percentile(sm,75)) #상위 25%     
    q3_list.append(result_q3)
    
    result_mid = print_want_val(sm, lambda x: np.percentile(sm,50)) # 중간값 반환 
    mid_list.append(result_mid)
    
    result_q1 = print_want_val(sm, lambda x: np.percentile(sm,25)) # 상위 75%
    q1_list.append(result_q1)

    print("\n")

1번째 리뷰어: gombi**


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Model Saved
최대값: 0.9878937910291989
상위25%: 0.986068052387207
중간값: 0.9827623543841908
상위75%: 0.9817729985933071
평균: 0.9838823459013638


2번째 리뷰어: sm81**
Model Saved
최대값: 0.44450565817164744
상위25%: 0.4349915386012617
중간값: 0.425477419030876
상위75%: 0.2736177388193564
평균: 0.3305803786034534


3번째 리뷰어: mj75**
Model Saved
최대값: 0.2484043025812012
상위25%: 0.23557278480467173
중간값: 0.22274126702814226
상위75%: 0.21083828844193692
평균: 0.223360293155025


4번째 리뷰어: cnh70**
Model Saved
최대값: 0.8383610480399518
상위25%: 0.8383610480399518
중간값: 0.8383610480399518
상위75%: 0.8383610480399518
평균: 0.8383610480399518


5번째 리뷰어: sjmhshym**
Model Saved
최대값: 0.989831433055917
상위25%: 0.984607170675143
중간값: 0.9823737357880367
상위75%: 0.9776503824933733
평균: 0.9808806009290914


6번째 리뷰어: sck090**
Model Saved
최대값: 0.9903475959472652
상위25%: 0.9903475959472652
중간값: 0.9903475959472652
상위75%: 0.9903475959472652
평균: 0.9903475959472652


7번째 리뷰어: sjy95**
Model Saved
최대값: 0.9653578215497621
상위25%: 0.43684908125832983
중간값: 0.34812

Model Saved
최대값: 0.22785775495175714
상위25%: 0.22785775495175714
중간값: 0.22785775495175714
상위75%: 0.22785775495175714
평균: 0.22785775495175714


55번째 리뷰어: loveo**
Model Saved
최대값: 0.9272896037601297
상위25%: 0.4422596807956864
중간값: 0.39336051937031274
상위75%: 0.30507180635117204
평균: 0.3962390530807744


56번째 리뷰어: bo92**
Model Saved
최대값: 0.2786602987627877
상위25%: 0.25771813433860147
중간값: 0.20965793345193454
상위75%: 0.17714029766351536
평균: 0.21370440492831766


57번째 리뷰어: ytzk1**
Model Saved
최대값: 0.9926259869733803
상위25%: 0.9920738188547271
중간값: 0.991521650736074
상위75%: 0.9912505528323018
평균: 0.991709030879328


58번째 리뷰어: doden3**
Model Saved
최대값: 0.5253942305510173
상위25%: 0.45026926209526386
중간값: 0.39956141607087814
상위75%: 0.3634800259438544
평균: 0.39984339476917025


59번째 리뷰어: uju06**
Model Saved
최대값: 0.9981508008052421
상위25%: 0.5614914458118359
중간값: 0.5191837911250521
상위75%: 0.4198727619226852
평균: 0.5460954950088399


60번째 리뷰어: ymj32**
Model Saved
최대값: 0.9940702179625539
상위25%: 0.9833893665629

In [188]:
test_df=pd.DataFrame(columns=['cId','pv_dm_max', 'pv_dm_q3','pv_dm_mid', 'pv_dm_q1'])
test_df['cId']=training_cId
test_df['pv_dm_max']=max_list
test_df['pv_dm_q3']=q3_list
test_df['pv_dm_mid']=mid_list
test_df['pv_dm_q1']=q1_list

test_df

,cId,pv_dm_max,pv_dm_q3,pv_dm_mid,pv_dm_q1
0,gombi**,0.9879,0.9861,0.9828,0.9818
1,sm81**,0.4445,0.4350,0.4255,0.2736
2,mj75**,0.2484,0.2356,0.2227,0.2108
3,cnh70**,0.8384,0.8384,0.8384,0.8384
4,sjmhshym**,0.9898,0.9846,0.9824,0.9777
5,sck090**,0.9903,0.9903,0.9903,0.9903
6,sjy95**,0.9654,0.4368,0.3481,0.2819
7,882663498512851**,0.6389,0.6389,0.6389,0.6389
8,yuhye8**,0.9112,0.4650,0.3770,0.3030
9,yeam**,0.8159,0.5930,0.3700,0.3047


In [211]:
def training_pvdbow(tagged_data):
    max_epochs = 100
    vec_size = 20
    alpha = 0.025

    model = Doc2Vec(size=vec_size,      # Dimensionality of the feature vectors
                window=8,            # The maximum distance 
                alpha=alpha,        # initial learning-rate
                min_alpha=0.00025,  # min learning-rate
                min_count=1,        # Ignores all words with total frequency lower than this.
                dm =0)              # pv-dbow

    model.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        #print('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha

    print("Model Saved")
    
    return model

In [194]:
max_list=[]
mid_list=[]
q1_list=[]
q3_list=[]

for i in range(len(training_cId)):
    print str(i+1) + str('번째 리뷰어: ')+ str(training_cId[i].encode('utf-8'))
    df=(go_data.loc[go_data['cId']==training_cId[i], ['cId','desc']])
    desc_per_cId = df["desc"].tolist()
    tagged_data = tagging(desc_per_cId)
    model = training_pvdbow(tagged_data)
    sm = similarity(model)
    
    result_max= print_want_val(sm, lambda x: np.percentile(sm,100)) # 최대값 반환 
    max_list.append(result_max)
       
    result_q3 = print_want_val(sm, lambda x: np.percentile(sm,75)) #상위 25%     
    q3_list.append(result_q3)
    
    result_mid = print_want_val(sm, lambda x: np.percentile(sm,50)) # 중간값 반환 
    mid_list.append(result_mid)
    
    result_q1 = print_want_val(sm, lambda x: np.percentile(sm,25)) # 상위 75%
    q1_list.append(result_q1)

    print("\n")

1번째 리뷰어: gombi**


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Model Saved
최대값: 0.815106303508862
상위25%: 0.7055104701662478
중간값: 0.6447172741724401
상위75%: 0.5800777292379963
평균: 0.6014852978448965


2번째 리뷰어: sm81**
Model Saved
최대값: 0.3147606085935588
상위25%: 0.31351492437192974
중간값: 0.31226924015030066
상위75%: 0.252397281187695
평균: 0.2731850569896496


3번째 리뷰어: mj75**
Model Saved
최대값: 0.2871696760746226
상위25%: 0.2869440115383326
중간값: 0.2867183470020425
상위75%: 0.26687128191174
평균: 0.2736374132993676


4번째 리뷰어: cnh70**
Model Saved
최대값: 0.2061331254394453
상위25%: 0.2061331254394453
중간값: 0.2061331254394453
상위75%: 0.2061331254394453
평균: 0.2061331254394453


5번째 리뷰어: sjmhshym**
Model Saved
최대값: 0.9837708629086587
상위25%: 0.9588845981340037
중간값: 0.840733926252978
상위75%: 0.7293241477505281
평균: 0.8102229429216338


6번째 리뷰어: sck090**
Model Saved
최대값: 0.7920418132956278
상위25%: 0.7920418132956278
중간값: 0.7920418132956278
상위75%: 0.7920418132956278
평균: 0.7920418132956278


7번째 리뷰어: sjy95**
Model Saved
최대값: 0.808958934693131
상위25%: 0.43456219004848273
중간값: 0.38871606

Model Saved
최대값: 0.12469412925948911
상위25%: 0.12469412925948911
중간값: 0.12469412925948911
상위75%: 0.12469412925948911
평균: 0.12469412925948911


55번째 리뷰어: loveo**
Model Saved
최대값: 0.6742868203039682
상위25%: 0.4506762730599087
중간값: 0.38588292267694824
상위75%: 0.33855479336164485
평균: 0.40174102390447824


56번째 리뷰어: bo92**
Model Saved
최대값: 0.35142304500549626
상위25%: 0.33393061658010104
중간값: 0.311089992254868
상위75%: 0.30309423561301546
평균: 0.31913193936060086


57번째 리뷰어: ytzk1**
Model Saved
최대값: 0.6210595329335339
상위25%: 0.5887483947050076
중간값: 0.5564372564764812
상위75%: 0.47921595538438994
평균: 0.5264971479007713


58번째 리뷰어: doden3**
Model Saved
최대값: 0.5347549457763944
상위25%: 0.4353166541565112
중간값: 0.3982274031325035
상위75%: 0.3595517254626526
평균: 0.39625251122852084


59번째 리뷰어: uju06**
Model Saved
최대값: 0.9983919795753606
상위25%: 0.41993720114808214
중간값: 0.38672566214323095
상위75%: 0.3107516757642161
평균: 0.4089879335936044


60번째 리뷰어: ymj32**
Model Saved
최대값: 0.4806104094304759
상위25%: 0.4101520308

In [183]:
len(training_cId)

100

In [195]:
test_df['pv_dbow_max']=max_list
test_df['pv_dbow_q3']=q3_list
test_df['pv_dbow_mid']=mid_list
test_df['pv_dbow_q1']=q1_list

test_df

,cId,pv_dm_max,pv_dm_q3,pv_dm_mid,pv_dm_q1,pv_dbow_max,pv_dbow_q3,pv_dbow_mid,pv_dbow_q1
0,gombi**,0.9879,0.9861,0.9828,0.9818,0.8151,0.7055,0.6447,0.5801
1,sm81**,0.4445,0.4350,0.4255,0.2736,0.3148,0.3135,0.3123,0.2524
2,mj75**,0.2484,0.2356,0.2227,0.2108,0.2872,0.2869,0.2867,0.2669
3,cnh70**,0.8384,0.8384,0.8384,0.8384,0.2061,0.2061,0.2061,0.2061
4,sjmhshym**,0.9898,0.9846,0.9824,0.9777,0.9838,0.9589,0.8407,0.7293
5,sck090**,0.9903,0.9903,0.9903,0.9903,0.7920,0.7920,0.7920,0.7920
6,sjy95**,0.9654,0.4368,0.3481,0.2819,0.8090,0.4346,0.3887,0.3349
7,882663498512851**,0.6389,0.6389,0.6389,0.6389,0.1442,0.1442,0.1442,0.1442
8,yuhye8**,0.9112,0.4650,0.3770,0.3030,0.7348,0.4875,0.4011,0.3461
9,yeam**,0.8159,0.5930,0.3700,0.3047,0.5015,0.3561,0.2108,0.2032


In [196]:
df=(go_data.loc[go_data['cId']=='gombi**', ['cId','desc']])
desc_per_cId = df["desc"].tolist()
tagged_data = tagging(desc_per_cId)
model = training_pvdbow(tagged_data)
sm = similarity(model)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Model Saved
최대값: 0.815106303508862
상위25%: 0.7055104701662478
중간값: 0.6447172741724401
상위75%: 0.5800777292379963
평균: 0.6014852978448965


In [200]:
len(model.docvecs)

4

In [201]:
sm

[0.815106303508862,
 0.6655424559824026,
 0.5654729415298357,
 0.718833141560863,
 0.6238920923624777,
 0.22006485212493773]

# 전체 데이터 - nltk

In [202]:
len(cIds_list)

92802

In [217]:
max_list=[]
mid_list=[]
q1_list=[]
q3_list=[]

for i in range(len(cIds_list)):
    print str(i+1) + str('번째 리뷰어: ')+ str(cIds_list[i].encode('utf-8'))
    df=(go_data.loc[go_data['cId']==cIds_list[i], ['cId','desc']])
    desc_per_cId = df["desc"].tolist()
    tagged_data = tokenizer_twitter_morphs(desc_per_cId)
    model = training_pvdbow(tagged_data)
    sm = similarity(model)
    
    result_max= print_want_val(sm, lambda x: np.percentile(sm,100)) # 최대값 반환 
    max_list.append(result_max)
       
    result_q3 = print_want_val(sm, lambda x: np.percentile(sm,75)) #상위 25%     
    q3_list.append(result_q3)
    
    result_mid = print_want_val(sm, lambda x: np.percentile(sm,50)) # 중간값 반환 
    mid_list.append(result_mid)
    
    result_q1 = print_want_val(sm, lambda x: np.percentile(sm,25)) # 상위 75%
    q1_list.append(result_q1)

    print("\n")

1번째 리뷰어: rkdldk5**


RuntimeError: No matching overloads found for tokenize in find. at native/common/jp_method.cpp:127

In [209]:
print len(max_list)
print len(q3_list)
print len(mid_list)
print len(q1_list)
print len(cIds_list)

92802
92802
92802
92802
92802


In [210]:
result_df=pd.DataFrame(columns=['cId','max', 'q3','mid', 'q1'])
result_df['cId']=cIds_list
result_df['max']=max_list
result_df['q3']=q3_list
result_df['mid']=mid_list
result_df['q1']=q1_list

result_df

,cId,max,q3,mid,q1
0,rkdldk5**,0.9996,0.4801,0.3707,0.2648
1,yeji04**,0.9992,0.4756,0.3747,0.2743
2,elqxhs**,0.9995,0.4679,0.3664,0.2656
3,dnv**,0.9996,0.4650,0.3638,0.2622
4,rladmswl9708**,0.8356,0.4630,0.3616,0.2601
5,wlgus5**,0.9995,0.4963,0.3994,0.2999
6,pp**,0.9992,0.4779,0.3787,0.2782
7,wertyuio9**,0.9990,0.4864,0.3902,0.2902
8,sumin12**,0.9998,0.6037,0.5173,0.4251
9,dldmsql10**,0.9994,0.4532,0.3518,0.2504


# 전체데이터 - twitter

In [219]:
df

,cId,desc
127,rkdldk5**,블랙구매했구요올라오자마자 구매했어요ㅋㅋ아직 입어보진않았지만새냄새가 좀 많이나서 바로빨았어요
128,rkdldk5**,쿨 레깅스라길래 샀어요편한차림좋아하는데같이주문한 박시반티셔츠에 같이입음딱일것같네요잘 구매한것같습니다!
129,rkdldk5**,그레이 주문했어요화이트랑 블랙이 입고지연이라그레이했는데그래도 빨리받은것같아요레깅스에 편하게입기좋은 티 찾고있었는데발견해서 좋네요!
130,rkdldk5**,오래기다렸네요ㅋㅋ오늘 받았어요여름에 너무최척화된 옷인것같아요바닷가에도 잘어울릴것같아요
131,rkdldk5**,6월25일에주문했는데 2주넘게걸려서받았네요오래기다린만큼 이상없이 잘왔구예뻐서 좋아요ㅜㅜ입어보니 잘 맞아서 좋구요자주입을것같아요
132,rkdldk5**,머스타드색상에 가깝네요ㅋㅋ청바지에 입으려고주문했어요뒷리본이 포인트라 좋을것같구시원한 재질인것같아요
940,rkdldk5**,딥블루구매했어요맨날 어두운것만입다가그래도 색이 좀 있는거로 구매했더니여름색상으로좋은것같아요
941,rkdldk5**,연청 구매했어요일자핏이라 그런지 불편하지않아좋아요하체가 커버되서 좋네요ㅋㅋ연청이없어서 샀는데 자주입을것같아요
942,rkdldk5**,블랙이 하나있어서베이지로 구매했어요신축성이 없어서 입고벗을때 쬐끔 불편하긴한데입을만해요얇고 여름에입고다니기딱이네요ㅋㅋ
943,rkdldk5**,그레이구매했어요원피스나 멜빵바지안에 입으려고 구매했어요저렴하게 구매한것같아서 넘 좋아요ㅋㅋ


In [221]:
df['token_review'] = df['desc'].apply(tokenizer_twitter_morphs)
df

,cId,desc,token_review
127,rkdldk5**,블랙구매했구요올라오자마자 구매했어요ㅋㅋ아직 입어보진않았지만새냄새가 좀 많이나서 바로빨았어요,"[블랙, 구매, 했, 구요, 올라오, 자마자, 구매, 했, 어요, ㅋㅋ, 아직, 입어보진, 않았, 지만, 새, 냄새, 가, 좀, 많이, 나서, 바로, 빨, 았, 어요]"
128,rkdldk5**,쿨 레깅스라길래 샀어요편한차림좋아하는데같이주문한 박시반티셔츠에 같이입음딱일것같네요잘 구매한것같습니다!,"[쿨, 레깅스, 라, 길, 래, 샀, 어요, 편한, 차림, 좋아하는, 데, 같이, 주문한, 박, 시반, 티셔츠, 에, 같이, 입, 음, 딱, 일, 것, 같네, 요, 잘, 구매, 한, 것, 같, 습니다, !]"
129,rkdldk5**,그레이 주문했어요화이트랑 블랙이 입고지연이라그레이했는데그래도 빨리받은것같아요레깅스에 편하게입기좋은 티 찾고있었는데발견해서 좋네요!,"[그레이, 주문했, 어요, 화이트, 랑, 블랙, 이, 입고, 지연, 이라, 그레이, 했, 는데, 그래도, 빨리, 받은, 것, 같아, 요, 레깅스, 에, 편하게, 입기, 좋, 은, 티, 찾고, 있, 었, 는데, 발견해서, 좋, 네요, !]"
130,rkdldk5**,오래기다렸네요ㅋㅋ오늘 받았어요여름에 너무최척화된 옷인것같아요바닷가에도 잘어울릴것같아요,"[오래, 기다렸, 네요, ㅋㅋ, 오늘, 받았, 어요, 여름, 에, 너무, 최척, 화, 된, 옷, 인, 것, 같아, 요, 바닷가, 에도, 잘, 어울릴, 것, 같아, 요]"
131,rkdldk5**,6월25일에주문했는데 2주넘게걸려서받았네요오래기다린만큼 이상없이 잘왔구예뻐서 좋아요ㅜㅜ입어보니 잘 맞아서 좋구요자주입을것같아요,"[6, 월, 25, 일, 에, 주문했, 는데, 2, 주, 넘게, 걸려, 서, 받았, 네요, 오래, 기다린, 만큼, 이상, 없이, 잘, 왔, 구, 예뻐, 서, 좋, 아요, ㅜㅜ, 입어보니, 잘, 맞아, 서, 좋, 구요자, 주입, 을, 것, 같아, 요]"
132,rkdldk5**,머스타드색상에 가깝네요ㅋㅋ청바지에 입으려고주문했어요뒷리본이 포인트라 좋을것같구시원한 재질인것같아요,"[머스타드, 색상, 에, 가깝, 네요, ㅋㅋ, 청바지, 에, 입으, 려고, 주문했, 어요, 뒷, 리본, 이, 포인트, 라, 좋, 을, 것, 같구, 시원한, 재질, 인, 것, 같아, 요]"
940,rkdldk5**,딥블루구매했어요맨날 어두운것만입다가그래도 색이 좀 있는거로 구매했더니여름색상으로좋은것같아요,"[딥블루, 구매, 했, 어요, 맨날, 어, 두, 운, 것, 만, 입다, 가, 그래도, 색, 이, 좀, 있는, 거, 로, 구매, 했, 더니, 여름, 색상, 으로, 좋, 은, 것, 같아, 요]"
941,rkdldk5**,연청 구매했어요일자핏이라 그런지 불편하지않아좋아요하체가 커버되서 좋네요ㅋㅋ연청이없어서 샀는데 자주입을것같아요,"[연청, 구매, 했, 어요, 일자, 핏, 이라, 그런, 지, 불편하지, 않아, 좋, 아요, 하체, 가, 커버, 되, 서, 좋, 네요, ㅋㅋ, 연청, 이, 없어, 서, 샀, 는데, 자, 주입, 을, 것, 같아, 요]"
942,rkdldk5**,블랙이 하나있어서베이지로 구매했어요신축성이 없어서 입고벗을때 쬐끔 불편하긴한데입을만해요얇고 여름에입고다니기딱이네요ㅋㅋ,"[블랙, 이, 하나, 있어, 서, 베이지, 로, 구매, 했, 어요, 신축, 성, 이, 없어, 서, 입고, 벗, 을, 때, 쬐끔, 불편하긴, 한데, 입, 을, 만해, 요, 얇고, 여름, 에, 입고, 다, 니, 기, 딱, 이, 네요, ㅋㅋ]"
943,rkdldk5**,그레이구매했어요원피스나 멜빵바지안에 입으려고 구매했어요저렴하게 구매한것같아서 넘 좋아요ㅋㅋ,"[그레이, 구매, 했, 어요, 원피스, 나, 멜빵, 바지, 안, 에, 입으, 려고, 구매, 했, 어요, 저렴, 하게, 구매, 한, 것, 같아, 서, 넘, 좋, 아요, ㅋㅋ]"


In [222]:
max_list=[]
mid_list=[]
q1_list=[]
q3_list=[]

for i in range(len(training_cId)):
    print str(i+1) + str('번째 리뷰어: ')+ str(training_cId[i].encode('utf-8'))
    df=(go_data.loc[go_data['cId']==training_cId[i], ['cId','token_review']])
    desc_per_cId = df["token_review"].tolist()
    tagged_data = tagging(desc_per_cId)
    model = training(tagged_data)
    sm = similarity(model)
    
    result_max= print_want_val(sm, lambda x: np.percentile(sm,100)) # 최대값 반환 
    max_list.append(result_max)
       
    result_q3 = print_want_val(sm, lambda x: np.percentile(sm,75)) #상위 25%     
    q3_list.append(result_q3)
    
    result_mid = print_want_val(sm, lambda x: np.percentile(sm,50)) # 중간값 반환 
    mid_list.append(result_mid)
    
    result_q1 = print_want_val(sm, lambda x: np.percentile(sm,25)) # 상위 75%
    q1_list.append(result_q1)

    print("\n")

1번째 리뷰어: gombi**


AttributeError: 'float' object has no attribute 'words'